In [2]:
whole_code = """
from enum import Enum
from typing import Any, Dict, Literal, Optional

from pydantic import SecretStr
from requests.exceptions import RequestException

from backend.data.block import Block, BlockCategory, BlockOutput, BlockSchema
from backend.data.model import (
    APIKeyCredentials,
    CredentialsField,
    CredentialsMetaInput,
    SchemaField,
)
from backend.integrations.providers import ProviderName
from backend.util.request import requests

TEST_CREDENTIALS = APIKeyCredentials(
    id="01234567-89ab-cdef-0123-456789abcdef",
    provider="ideogram",
    api_key=SecretStr("mock-ideogram-api-key"),
    title="Mock Ideogram API key",
    expires_at=None,
)
TEST_CREDENTIALS_INPUT = {
    "provider": TEST_CREDENTIALS.provider,
    "id": TEST_CREDENTIALS.id,
    "type": TEST_CREDENTIALS.type,
    "title": TEST_CREDENTIALS.type,
}


class IdeogramModelName(str, Enum):
    V2 = "V_2"
    V1 = "V_1"
    V1_TURBO = "V_1_TURBO"
    V2_TURBO = "V_2_TURBO"


class MagicPromptOption(str, Enum):
    AUTO = "AUTO"
    ON = "ON"
    OFF = "OFF"


class StyleType(str, Enum):
    AUTO = "AUTO"
    GENERAL = "GENERAL"
    REALISTIC = "REALISTIC"
    DESIGN = "DESIGN"
    RENDER_3D = "RENDER_3D"
    ANIME = "ANIME"


class ColorPalettePreset(str, Enum):
    NONE = "NONE"
    EMBER = "EMBER"
    FRESH = "FRESH"
    JUNGLE = "JUNGLE"
    MAGIC = "MAGIC"
    MELON = "MELON"
    MOSAIC = "MOSAIC"
    PASTEL = "PASTEL"
    ULTRAMARINE = "ULTRAMARINE"


class AspectRatio(str, Enum):
    ASPECT_10_16 = "ASPECT_10_16"
    ASPECT_16_10 = "ASPECT_16_10"
    ASPECT_9_16 = "ASPECT_9_16"
    ASPECT_16_9 = "ASPECT_16_9"
    ASPECT_3_2 = "ASPECT_3_2"
    ASPECT_2_3 = "ASPECT_2_3"
    ASPECT_4_3 = "ASPECT_4_3"
    ASPECT_3_4 = "ASPECT_3_4"
    ASPECT_1_1 = "ASPECT_1_1"
    ASPECT_1_3 = "ASPECT_1_3"
    ASPECT_3_1 = "ASPECT_3_1"


class UpscaleOption(str, Enum):
    AI_UPSCALE = "AI Upscale"
    NO_UPSCALE = "No Upscale"


class IdeogramModelBlock(Block):
    class Input(BlockSchema):
        credentials: CredentialsMetaInput[
            Literal[ProviderName.IDEOGRAM], Literal["api_key"]
        ] = CredentialsField(
            description="The Ideogram integration can be used with any API key with sufficient permissions for the blocks it is used on.",
        )
        prompt: str = SchemaField(
            description="Text prompt for image generation",
            placeholder="e.g., 'A futuristic cityscape at sunset'",
            title="Prompt",
        )
        ideogram_model_name: IdeogramModelName = SchemaField(
            description="The name of the Image Generation Model, e.g., V_2",
            default=IdeogramModelName.V2,
            title="Image Generation Model",
            advanced=False,
        )
        aspect_ratio: AspectRatio = SchemaField(
            description="Aspect ratio for the generated image",
            default=AspectRatio.ASPECT_1_1,
            title="Aspect Ratio",
            advanced=False,
        )
        upscale: UpscaleOption = SchemaField(
            description="Upscale the generated image",
            default=UpscaleOption.NO_UPSCALE,
            title="Upscale Image",
            advanced=False,
        )
        magic_prompt_option: MagicPromptOption = SchemaField(
            description="Whether to use MagicPrompt for enhancing the request",
            default=MagicPromptOption.AUTO,
            title="Magic Prompt Option",
            advanced=True,
        )
        seed: Optional[int] = SchemaField(
            description="Random seed. Set for reproducible generation",
            default=None,
            title="Seed",
            advanced=True,
        )
        style_type: StyleType = SchemaField(
            description="Style type to apply, applicable for V_2 and above",
            default=StyleType.AUTO,
            title="Style Type",
            advanced=True,
        )
        negative_prompt: Optional[str] = SchemaField(
            description="Description of what to exclude from the image",
            default=None,
            title="Negative Prompt",
            advanced=True,
        )
        color_palette_name: ColorPalettePreset = SchemaField(
            description="Color palette preset name, choose 'None' to skip",
            default=ColorPalettePreset.NONE,
            title="Color Palette Preset",
            advanced=True,
        )

    class Output(BlockSchema):
        result: str = SchemaField(description="Generated image URL")
        error: str = SchemaField(description="Error message if the model run failed")

    def __init__(self):
        super().__init__(
            id="6ab085e2-20b3-4055-bc3e-08036e01eca6",
            description="This block runs Ideogram models with both simple and advanced settings.",
            categories={BlockCategory.AI},
            input_schema=IdeogramModelBlock.Input,
            output_schema=IdeogramModelBlock.Output,
            test_input={
                "ideogram_model_name": IdeogramModelName.V2,
                "prompt": "A futuristic cityscape at sunset",
                "aspect_ratio": AspectRatio.ASPECT_1_1,
                "upscale": UpscaleOption.NO_UPSCALE,
                "magic_prompt_option": MagicPromptOption.AUTO,
                "seed": None,
                "style_type": StyleType.AUTO,
                "negative_prompt": None,
                "color_palette_name": ColorPalettePreset.NONE,
                "credentials": TEST_CREDENTIALS_INPUT,
            },
            test_output=[
                (
                    "result",
                    "https://ideogram.ai/api/images/test-generated-image-url.png",
                ),
            ],
            test_mock={
                "run_model": lambda api_key, model_name, prompt, seed, aspect_ratio, magic_prompt_option, style_type, negative_prompt, color_palette_name: "https://ideogram.ai/api/images/test-generated-image-url.png",
                "upscale_image": lambda api_key, image_url: "https://ideogram.ai/api/images/test-upscaled-image-url.png",
            },
            test_credentials=TEST_CREDENTIALS,
        )

    def run(
        self, input_data: Input, *, credentials: APIKeyCredentials, **kwargs
    ) -> BlockOutput:
        seed = input_data.seed

        # Step 1: Generate the image
        result = self.run_model(
            api_key=credentials.api_key,
            model_name=input_data.ideogram_model_name.value,
            prompt=input_data.prompt,
            seed=seed,
            aspect_ratio=input_data.aspect_ratio.value,
            magic_prompt_option=input_data.magic_prompt_option.value,
            style_type=input_data.style_type.value,
            negative_prompt=input_data.negative_prompt,
            color_palette_name=input_data.color_palette_name.value,
        )

        # Step 2: Upscale the image if requested
        if input_data.upscale == UpscaleOption.AI_UPSCALE:
            result = self.upscale_image(
                api_key=credentials.api_key,
                image_url=result,
            )

        yield "result", result

    def run_model(
        self,
        api_key: SecretStr,
        model_name: str,
        prompt: str,
        seed: Optional[int],
        aspect_ratio: str,
        magic_prompt_option: str,
        style_type: str,
        negative_prompt: Optional[str],
        color_palette_name: str,
    ):
        url = "https://api.ideogram.ai/generate"
        headers = {
            "Api-Key": api_key.get_secret_value(),
            "Content-Type": "application/json",
        }

        data: Dict[str, Any] = {
            "image_request": {
                "prompt": prompt,
                "model": model_name,
                "aspect_ratio": aspect_ratio,
                "magic_prompt_option": magic_prompt_option,
                "style_type": style_type,
            }
        }

        if seed is not None:
            data["image_request"]["seed"] = seed

        if negative_prompt:
            data["image_request"]["negative_prompt"] = negative_prompt

        if color_palette_name != "NONE":
            data["image_request"]["color_palette"] = {"name": color_palette_name}

        try:
            response = requests.post(url, json=data, headers=headers)
            return response.json()["data"][0]["url"]
        except RequestException as e:
            raise Exception(f"Failed to fetch image: {str(e)}")

    def upscale_image(self, api_key: SecretStr, image_url: str):
        url = "https://api.ideogram.ai/upscale"
        headers = {
            "Api-Key": api_key.get_secret_value(),
        }

        try:
            # Step 1: Download the image from the provided URL
            image_response = requests.get(image_url)

            # Step 2: Send the downloaded image to the upscale API
            files = {
                "image_file": ("image.png", image_response.content, "image/png"),
            }

            response = requests.post(
                url,
                headers=headers,
                data={
                    "image_request": "{}",  # Empty JSON object
                },
                files=files,
            )

            return response.json()["data"][0]["url"]

        except RequestException as e:
            raise Exception(f"Failed to upscale image: {str(e)}")
"""




In [4]:
import json
from typing import List, Dict, Any
from llm import get_llm_answers

def step_0_split_top_declarations(whole_code: str, logger) -> List[Dict[str, Any]]:
    """
    Step 0: 对大代码做解析,将顶层声明按行号拆分.
    
    在此处明确要求：如果类/函数中出现嵌套函数/方法，也要拆分为独立块。
    """
    try:
        logger.info("Starting Step 0: Splitting code into top-level declarations")
        
        code_lines = whole_code.split('\n')
        code_array = [{"line": i, "content": line} for i, line in enumerate(code_lines)]
        
        # 修改提示，强调“嵌套函数/方法”也单独作为块，不与外层块有顺序关系
        prompt = """
You are a senior code analysis assistant. We have a potentially large Python code snippet.  
We want to split it into a hierarchical JSON structure where each declaration or block can contain its own `children` if nested structures exist.

### Requirements:
1) Each element in the JSON array should be an object with:
   - `"decl_name"`: an identifier or best guess of the block's name.
   - `"start_line"` and `"end_line"`: line numbers in the original snippet.
   - `"children"`: an array of **nested blocks** if the block contains nested methods, functions, or other identifiable structures.

2) Every line from the code must appear in exactly one block (covering the entire file).

3) If there are top-level statements not inside any class or function, group them into a block called `"GlobalBlock"`. This block should also contain nested children for any identifiable blocks.

4) **Important**: 
   - If inside a class or function we detect a nested function/method, 
     treat that nested function/method as a separate block and include it in the `children` array of its parent block.
   - Nested blocks should themselves be fully structured as blocks, with their own `"decl_name"`, `"start_line"`, `"end_line"`, and potentially further `"children"`.
   - In later steps, there should be no direct 'sequential' edges between the parent block and the nested block; they should be treated independently.

5) Blocks must capture the hierarchical relationships of the code. For example:
   - A `class` contains `methods` as its children.
   - A `method` may contain nested functions as its children.

6) The final response must be strictly valid JSON. Example:

```json
{
  "blocks": [
    {
      "decl_name": "GlobalBlock",
      "start_line": 1,
      "end_line": 5,
      "children": []
    },
    {
      "decl_name": "Apple",
      "start_line": 6,
      "end_line": 18,
      "children": [
        {
          "decl_name": "Apple.m",
          "start_line": 11,
          "end_line": 12,
          "children": []
        },
        {
          "decl_name": "Apple.plant",
          "start_line": 14,
          "end_line": 18,
          "children": [
            {
              "decl_name": "Apple.plant.process_single",
              "start_line": 16,
              "end_line": 17,
              "children": []
            }
          ]
        }
      ]
    }
  ]
}
7. Do not output any extra text or code besides the JSON.
Python code (with line numbers):
""" + json.dumps(code_array, indent=2)

        response = get_llm_answers(prompt, model_name="gpt-4o", require_json=True)
        chunks = json.loads(response)["blocks"]
        # 进一步处理每个块，生成 code
        def process_code_blocks(whole_code: str, blocks: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
            """
            进一步处理 blocks，生成每个块的完整 "code"。
            - 如果没有 children，则直接使用 start_line 到 end_line 的代码。
            - 如果有 children，则去掉 children 的部分，保留其余部分作为 code。
            """
            def extract_code(lines: List[str], start: int, end: int, children: List[Dict[str, Any]]) -> str:
                """
                根据起始行和结束行提取代码，并去掉子块的代码部分。
                """
                # 用于标记每行是否被子块占用
                excluded_lines = set()
                for child in children:
                    excluded_lines.update(range(child["start_line"], child["end_line"] + 1))
                
                # 提取属于当前块但不属于子块的代码行
                filtered_lines = [
                    line for i, line in enumerate(lines[start:end + 1], start=start) if i not in excluded_lines
                ]
                return "\n".join(filtered_lines) + "\n"

            # 将整个代码按行拆分
            code_lines = whole_code.split('\n')

            # 遍历每个块并生成对应的 code
            def process_block(block: Dict[str, Any]):
                start_line = block["start_line"]
                end_line = block["end_line"]
                children = block.get("children", [])

                if not children:
                    # 如果没有子块，直接取 start_line 到 end_line 的代码
                    block["code"] = "\n".join(code_lines[start_line:end_line + 1]) + "\n"
                else:
                    # 如果有子块，去掉子块的代码部分
                    block["code"] = extract_code(code_lines, start_line, end_line, children)

                    # 递归处理子块
                    for child in children:
                        process_block(child)

            # 遍历所有顶层块，处理每个块及其子块
            for block in blocks:
                process_block(block)

            return blocks

        # 调用处理函数
        processed_chunks = process_code_blocks(whole_code, chunks)

        logger.info(f"Successfully split code into {len(processed_chunks)} top-level declarations")
        return processed_chunks
    except Exception as e:
        logger.error(f"Error in step_0_split_top_declarations: {str(e)}")
        raise
    

def step_1_basic_block_segmentation(code: str, logger) -> List[Dict[str, Any]]:
    """
    Step 1: 对每个代码块做解析,将代码块按行号拆分.
    """
    try:
        logger.info("Starting Step 1: Basic block segmentation")
        
        code_lines = code.split('\n')
        code_array = [{"line": i, "content": line} for i, line in enumerate(code_lines)]

        prompt = """
Please analyze the following code and identify its basic blocks. 
Return the start and end line numbers for each basic block in JSON format.

Input code:
""" + json.dumps(code_array, indent=2) + """

Expected output format:
{
    "basic_blocks": [
        {
            "start_line": x,
            "end_line": y
        },
        ...
    ]
}

A basic block is a sequence of consecutive statements where:
- Flow of control enters at the beginning
- Flow of control leaves at the end
- No branching occurs except at the end
- No branching targets exist except at the beginning

Do not include any extra explanation, just the JSON.
"""
        response = get_llm_answers(prompt, model_name="deepseek-chat", require_json=True)
        basic_blocks = json.loads(response)["basic_blocks"]
        
        # 将分好的块加上实际code
        for block in basic_blocks:
            start_line = block["start_line"]
            end_line = block["end_line"]
            block["code"] = "\n".join(code_lines[start_line:end_line+1]) + "\n"
            
        logger.info(f"Successfully segmented code into {len(basic_blocks)} basic blocks")
        return basic_blocks
    except Exception as e:
        logger.error(f"Error in step_1_basic_block_segmentation: {str(e)}")
        raise

def step_2_determine_execution_order(basic_blocks: List[Dict[str, Any]], code: str, logger) -> List[Dict[str, Any]]:
    """
    Step 2: 根据基本块确定它们之间的控制流关系.
    
    在此处再度强调：如果块代表的是一个函数/方法与外层互为嵌套，则彼此不应当有顺序边。
    同一类下多个方法也不连边，除非显式调用。
    """
    try:
        logger.info("Starting Step 2: Determining execution order")
        
        blocks_content = [
            {"block_id": idx, "code": block["code"]} 
            for idx, block in enumerate(basic_blocks, 1)
        ]
        
        # 修改提示，强调嵌套函数/方法为独立块，不与外层连顺序边
        prompt = """
You are a senior code analysis assistant. Based on the provided basic blocks, determine the control flow between them.

Each basic block is identified by a unique "block_id" and has some code.

### Requirements:
1) If a block is recognized as a separate (nested) function/method, do NOT add a control flow edge 
   from the outer function/class to this block (nor vice versa).
2) Similarly, if multiple methods appear within the same class, do NOT have a direct execution flow 
   unless there's an explicit call or branching in the code.
3) For normal sequential blocks within the same function, link them accordingly in the typical order.

Return the result in strictly valid JSON:
{
    "control_flow": [
        {
            "block_id": 1,
            "successors": [2, 3]
        },
        ...
    ]
}

Do not include any extra text or explanations.

Original code:
""" + code + """

Basic Blocks:
""" + json.dumps(blocks_content, indent=2) + """
"""
        response = get_llm_answers(prompt, model_name="deepseek-chat", require_json=True)
        control_flow = json.loads(response)["control_flow"]
        logger.info(f"Successfully determined execution order for {len(control_flow)} blocks")
        return control_flow
    except Exception as e:
        logger.error(f"Error in step_2_determine_execution_order: {str(e)}")
        raise

def step_3_generate_cfg(basic_blocks: List[Dict[str, Any]], control_flow: List[Dict[str, Any]], decl_name: str, logger) -> Dict[str, Any]:
    """
    Step 3: 根据基本块和控制流关系生成CFG.
    """
    try:
        logger.info("Starting Step 3: Generating CFG")
        
        # 生成节点
        nodes = [
            {"id": f"{decl_name}_{idx}", "code": block["code"]}
            for idx, block in enumerate(basic_blocks, 1)
        ]
        
        # 生成边
        edges = []
        for flow in control_flow:
            from_id = f"{decl_name}_{flow['block_id']}"
            for to_id in flow["successors"]:
                edges.append({
                    "from": from_id,
                    "to": f"{decl_name}_{to_id}"
                })
        
        cfg = {"nodes": nodes, "edges": edges}
        logger.info(f"Successfully generated CFG with {len(nodes)} nodes and {len(edges)} edges")
        return cfg
    except Exception as e:
        logger.error(f"Error in step_3_generate_cfg: {str(e)}")
        raise

def parse_single_block(block: Dict[str, Any], logger) -> None:
    """
    对单个 block 的 'code' 调用 3 个步骤，生成 CFG 并存入 block["cfg"].
    如果该 block 的 code 为空或无实际内容，可根据需求选择是否跳过.
    """
    code_str = block.get("code", "").strip()
    if not code_str:
        # 若 code 为空，可根据需求决定跳不跳过
        logger.info(f"Block '{block.get('decl_name')}' has empty code, skipped.")
        return
    
    # 第 1 步: 进行基本块划分
    basic_blocks = step_1_basic_block_segmentation(code_str, logger)
    
    # 第 2 步: 确定控制流关系
    control_flow = step_2_determine_execution_order(basic_blocks, code_str, logger)
    
    # 第 3 步: 生成 CFG
    cfg = step_3_generate_cfg(basic_blocks, control_flow, block.get("decl_name", ""), logger)
    
    # 将生成的 CFG 存到该 block 的字段中
    block["cfg"] = cfg

def parse_blocks_recursively(blocks: List[Dict[str, Any]], logger) -> None:
    """
    递归地遍历所有 block 及其 children，
    对每个 block 生成 CFG 并储存到 block["cfg"]。
    """
    for block in blocks:
        # 先处理当前 block
        parse_single_block(block, logger)
        
        # 如果有子 block，则递归处理
        children = block.get("children", [])
        if children:
            parse_blocks_recursively(children, logger)

In [5]:
import logging

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger("CFG_Logger")
    chunks = step_0_split_top_declarations(whole_code, logger)
    parse_blocks_recursively(chunks, logger)

    for block in chunks:
        decl_name = block.get("decl_name", "")
        cfg = block.get("cfg")
        logger.info(f"Block '{decl_name}' CFG: {json.dumps(cfg, indent=2)}")
        
        # 如果有 children，也可查看
        for child in block.get("children", []):
            child_name = child.get("decl_name", "")
            child_cfg = child.get("cfg")
            logger.info(f"  Child block '{child_name}' CFG: {json.dumps(child_cfg, indent=2)}")

INFO:CFG_Logger:Starting Step 0: Splitting code into top-level declarations
INFO:httpx:HTTP Request: POST https://xiaoai.plus/v1/chat/completions "HTTP/1.1 200 OK"
INFO:CFG_Logger:Successfully split code into 8 top-level declarations
INFO:CFG_Logger:Starting Step 1: Basic block segmentation
INFO:httpx:HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:CFG_Logger:Successfully segmented code into 6 basic blocks
INFO:CFG_Logger:Starting Step 2: Determining execution order
INFO:httpx:HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:CFG_Logger:Successfully determined execution order for 6 blocks
INFO:CFG_Logger:Starting Step 3: Generating CFG
INFO:CFG_Logger:Successfully generated CFG with 6 nodes and 5 edges
INFO:CFG_Logger:Starting Step 1: Basic block segmentation
INFO:httpx:HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:CFG_Logger:Successfully segmented code into 1 basic bloc

In [8]:
print(json.dumps(chunks, indent=2))

with open("refined_cfg.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, indent=2, ensure_ascii=False)

[
  {
    "decl_name": "GlobalBlock",
    "start_line": 1,
    "end_line": 31,
    "children": [],
    "code": "from enum import Enum\nfrom typing import Any, Dict, Literal, Optional\n\nfrom pydantic import SecretStr\nfrom requests.exceptions import RequestException\n\nfrom backend.data.block import Block, BlockCategory, BlockOutput, BlockSchema\nfrom backend.data.model import (\n    APIKeyCredentials,\n    CredentialsField,\n    CredentialsMetaInput,\n    SchemaField,\n)\nfrom backend.integrations.providers import ProviderName\nfrom backend.util.request import requests\n\nTEST_CREDENTIALS = APIKeyCredentials(\n    id=\"01234567-89ab-cdef-0123-456789abcdef\",\n    provider=\"ideogram\",\n    api_key=SecretStr(\"mock-ideogram-api-key\"),\n    title=\"Mock Ideogram API key\",\n    expires_at=None,\n)\nTEST_CREDENTIALS_INPUT = {\n    \"provider\": TEST_CREDENTIALS.provider,\n    \"id\": TEST_CREDENTIALS.id,\n    \"type\": TEST_CREDENTIALS.type,\n    \"title\": TEST_CREDENTIALS.type,\n}\n\

In [7]:
import json

class IDGenerator:
    """简单的自增 id 生成器"""
    def __init__(self):
        self._current_id = 0
    
    def next_id(self):
        self._current_id += 1
        return self._current_id

def build_level(block_data_list, id_gen):
    """
    处理“同一层级”的若干代码块，返回 (nodes, edges)：
      - nodes: 本层所有节点 (每个节点包含自身的 nodes/edges，用于递归嵌套)
      - edges: 只在本层兄弟节点之间建立 [i->i+1], 不建立“父->子”边
    """
    nodes = []
    edges = []
    
    # 1) 依次遍历本层的所有 block
    for block_data in block_data_list:
        # 生成当前块对应的节点
        node_id = id_gen.next_id()
        statements = block_data["code"].splitlines()
        node = {
            "id": node_id,
            "name": block_data.get("decl_name", ""),
            "type": "BasicBlock",
            "statements": statements,
            "nodes": [],
            "edges": []
        }
        
        # 2) 递归处理其子块（下一层），得到子节点和子层级的 edges
        child_blocks = block_data.get("children", [])
        child_nodes, child_edges = build_level(child_blocks, id_gen)
        
        # 3) 将子层级直接挂到 node["nodes"] 和 node["edges"]
        node["nodes"] = child_nodes
        node["edges"] = child_edges
        
        # 收集到本层节点列表
        nodes.append(node)
    
    # 4) 在本层建立兄弟节点之间的顺序边 (node[i] -> node[i+1])
    for i in range(len(nodes) - 1):
        edge = {
            "from": nodes[i]["id"],
            "to": nodes[i+1]["id"]
        }
        edges.append(edge)
    
    return nodes, edges

def build_cfg_from_split_data(split_code_data):
    """
    构建最外层的“File”节点 (id=0)，其下的 nodes 为 split_code_data 各块。
    不生成 [0->子] 的跨层边，只生成本层与子层的兄弟节点间的顺序边。
    """
    id_gen = IDGenerator()
    
    # 顶层虚拟节点
    top_level_node = {
        "id": 0,
        "name": "File",
        "type": "FileBlock",
        "statements": [],
        "nodes": [],
        "edges": []
    }
    
    # 递归处理最外层（split_code_data）——将其视为同一层级的若干 block
    child_nodes, child_edges = build_level(split_code_data, id_gen)
    
    # 放入顶层节点的 nodes/edges
    top_level_node["nodes"] = child_nodes
    top_level_node["edges"] = child_edges  # 注意：顶层本身不与子节点连边，但子节点彼此可能有边
    
    return top_level_node
# ---------------- 下面演示如何使用上述函数 ----------------
if __name__ == "__main__":
    
    cfg_result = build_cfg_from_split_data(chunks)
    
    # 输出为 JSON 字符串查看
    print(json.dumps(cfg_result, indent=2, ensure_ascii=False))

    with open("refined_cfg.json", "w", encoding="utf-8") as f:
        json.dump(cfg_result, f, indent=2, ensure_ascii=False)


{
  "id": 0,
  "name": "File",
  "type": "FileBlock",
  "statements": [],
  "nodes": [
    {
      "id": 1,
      "name": "GlobalBlock",
      "type": "BasicBlock",
      "statements": [
        "from enum import Enum",
        "from typing import Any, Dict, Literal, Optional",
        "",
        "from pydantic import SecretStr",
        "from requests.exceptions import RequestException",
        "",
        "from backend.data.block import Block, BlockCategory, BlockOutput, BlockSchema",
        "from backend.data.model import (",
        "    APIKeyCredentials,",
        "    CredentialsField,",
        "    CredentialsMetaInput,",
        "    SchemaField,",
        ")",
        "from backend.integrations.providers import ProviderName",
        "from backend.util.request import requests",
        "",
        "TEST_CREDENTIALS = APIKeyCredentials(",
        "    id=\"01234567-89ab-cdef-0123-456789abcdef\",",
        "    provider=\"ideogram\",",
        "    api_key=SecretStr(\"mock-